# Clustering Example

In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('cluster_example').getOrCreate()

In [0]:
dataset = spark.read.csv('/FileStore/tables/seeds_dataset.csv', inferSchema=True, header=True)

In [0]:
dataset.printSchema()

root
 |-- area: double (nullable = true)
 |-- perimeter: double (nullable = true)
 |-- compactness: double (nullable = true)
 |-- length_of_kernel: double (nullable = true)
 |-- width_of_kernel: double (nullable = true)
 |-- asymmetry_coefficient: double (nullable = true)
 |-- length_of_groove: double (nullable = true)



In [0]:
dataset.head(1)

Out[5]: [Row(area=15.26, perimeter=14.84, compactness=0.871, length_of_kernel=5.763, width_of_kernel=3.312, asymmetry_coefficient=2.221, length_of_groove=5.22)]

# Process Data

In [0]:
from pyspark.ml.feature import VectorAssembler, StandardScaler

In [0]:
dataset.columns

Out[7]: ['area',
 'perimeter',
 'compactness',
 'length_of_kernel',
 'width_of_kernel',
 'asymmetry_coefficient',
 'length_of_groove']

In [0]:
assembler = VectorAssembler(inputCols=dataset.columns, outputCol='features')

In [0]:
final_data = assembler.transform(dataset)

In [0]:
final_data.printSchema()

root
 |-- area: double (nullable = true)
 |-- perimeter: double (nullable = true)
 |-- compactness: double (nullable = true)
 |-- length_of_kernel: double (nullable = true)
 |-- width_of_kernel: double (nullable = true)
 |-- asymmetry_coefficient: double (nullable = true)
 |-- length_of_groove: double (nullable = true)
 |-- features: vector (nullable = true)



In [0]:
scaler = StandardScaler(
    inputCol='features',
    outputCol='scaledFeatures'
)

In [0]:
scaled_final_data = scaler.fit(final_data).transform(final_data)

In [0]:
scaled_final_data.head(1)

Out[14]: [Row(area=15.26, perimeter=14.84, compactness=0.871, length_of_kernel=5.763, width_of_kernel=3.312, asymmetry_coefficient=2.221, length_of_groove=5.22, features=DenseVector([15.26, 14.84, 0.871, 5.763, 3.312, 2.221, 5.22]), scaledFeatures=DenseVector([5.2445, 11.3633, 36.8608, 13.0072, 8.7685, 1.4772, 10.621]))]

# Model

In [0]:
from pyspark.ml.clustering import KMeans

In [0]:
kmeans = KMeans(
    featuresCol='scaledFeatures',
    k=3
)

In [0]:
model = kmeans.fit(scaled_final_data)

In [0]:
print('WSSSE')
print(model.summary.trainingCost)

WSSSE
428.6720009438313


In [0]:
centers = model.clusterCenters()

In [0]:
print(centers)

[array([ 4.93382436, 10.94691274, 37.30542404, 12.41332714,  8.60366812,
        1.82917353, 10.40106154]), array([ 6.35645488, 12.40730852, 37.41990178, 13.93860446,  9.7892399 ,
        2.41585013, 12.29286107]), array([ 4.06660859, 10.14191893, 35.84098009, 11.81592066,  7.52397236,
        3.1823335 , 10.39801233])]


In [0]:
model.transform(scaled_final_data).select('prediction').show()

+----------+
|prediction|
+----------+
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         1|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         2|
+----------+
only showing top 20 rows

